In [ ]:
import os
import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from keras import backend as K 
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print('Notebook run using keras:', keras.__version__)

In [ ]:
inet_model = inc_net.InceptionV3()

In [ ]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

In [ ]:
image_path = "./strawberry.jpg"

In [ ]:
images = transform_img_fn([image_path])
# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(images[0] / 2 + 0.5)
preds = inet_model.predict(images)
for x in decode_predictions(preds):
    print(x)

## Lime

In [ ]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

In [ ]:
img = transform_img_fn([image_path])[0]

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
%%time
explanation_img = explainer.explain_instance(img, inet_model.predict, top_labels=5, hide_color=0, num_samples=200)

### Labels

949 corresponds to the imagenet label for "strawberry."

You can replace 949 with the labels for 'chocolate sauce', 'trifle', etc. You can look up the numbers that correspond to those labels at https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a.

In [ ]:
from skimage.segmentation import mark_boundaries

In [ ]:
label_img = 949

In [ ]:
temp, mask = explanation_img.get_image_and_mask(label_img, positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation_img.get_image_and_mask(label_img, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))